# Ejemplo de uso

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [17]:
# Configuración
options = Options()
# options.add_argument("--headless")  # activalo si no querés ver la ventana
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# URL principal
url = "https://estadisticascabb.gesdeportiva.es/partido/rw7VcUphzJV__jNBe2vL4A==?a=1"
driver.get(url)

# Esperar el primer iframe que contiene toda la info
WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))

True

In [18]:
# Ahora estamos *dentro* del iframe general, buscamos el iframe específico del mapa de tiro
iframe_mapa = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='mapa-tiro']"))
)

# Hacemos switch al iframe del mapa de tiro
driver.switch_to.frame(iframe_mapa)

# Extraemos el HTML del iframe del mapa de tiro
html_mapa_tiro = driver.page_source
print(html_mapa_tiro[:1000]) 

<html lang="en"><head>
    <meta charset="utf-8">
    <title>Widgets de Gesdeportiva</title>

    <meta name="viewport" content="width=device-width,minimum-scale=1.0,initial-scale=1.0,maximum-scale=5.0,user-scalable=yes,viewport-fit=cover">
    <meta name="format-detection" content="telephone=no">
    <meta name="robots" content="noindex">
    <link rel="icon" href="/imagenes/favicon.png">

    <!-- Bootstrap CSS -->
    <link href="/css/bootstrap-5.2.0/bootstrap-gesweb.min.css" rel="stylesheet">
    <link href="/css/widget/min/0-global.min.css" rel="stylesheet">

    
    <link href="/fontawesome/css/fontawesome.css" rel="stylesheet">
    <link href="/fontawesome/css/brands.css" rel="stylesheet">
    <link href="/fontawesome/css/solid.css" rel="stylesheet">
    <link href="/fontawesome/css/v5-font-face.css" rel="stylesheet">

    
    <link href="/css/widget/min/partidos/partido-en-vivo.min.css" rel="stylesheet">

        <style>
            :root {
                --color-principal: 

In [20]:
from bs4 import BeautifulSoup

# Parsear el HTML del iframe del mapa de tiros
soup = BeautifulSoup(html_mapa_tiro, "html.parser")

# Buscar todos los íconos de tiros
tiros = soup.find_all("i", class_="ico-tiro")

print(f"Se encontraron {len(tiros)} tiros.")

# Mostrar detalles de los primeros 3 tiros
for tiro in tiros[:30]:
    print(tiro)
    print(tiro.get("class"))

Se encontraron 117 tiros.
<i class="ico-tiro fas fa-times colorAZUL" style="left: 36.72%; top: 43.75%;"></i>
['ico-tiro', 'fas', 'fa-times', 'colorAZUL']
<i class="ico-tiro fas fa-times colorAZUL" style="left: 36.72%; top: 36.68%;"></i>
['ico-tiro', 'fas', 'fa-times', 'colorAZUL']
<i class="ico-tiro far fa-circle colorAZUL" style="left: 38.77%; top: 47.20%;"></i>
['ico-tiro', 'far', 'fa-circle', 'colorAZUL']
<i class="ico-tiro fas fa-times colorAZUL" style="left: 26.07%; top: 36.35%;"></i>
['ico-tiro', 'fas', 'fa-times', 'colorAZUL']
<i class="ico-tiro far fa-circle colorROJO" style="left: 85.55%; top: 49.01%;"></i>
['ico-tiro', 'far', 'fa-circle', 'colorROJO']
<i class="ico-tiro far fa-circle colorAZUL" style="left: 36.62%; top: 35.53%;"></i>
['ico-tiro', 'far', 'fa-circle', 'colorAZUL']
<i class="ico-tiro fas fa-times colorROJO" style="left: 82.23%; top: 56.09%;"></i>
['ico-tiro', 'fas', 'fa-times', 'colorROJO']
<i class="ico-tiro fas fa-times colorAZUL" style="left: 35.35%; top: 70.

In [22]:
import pandas as pd

data = []

for tiro in tiros:
    clases = tiro.get("class", [])
    estilo = tiro.get("style", "")
    
    # Determinar si el tiro fue acertado o fallado
    if "fa-times" in clases:
        resultado = "fallado"
    elif "fa-circle" in clases:
        resultado = "acertado"
    else:
        resultado = "desconocido"
    
    # Determinar el equipo
    if "colorAZUL" in clases:
        equipo = "azul"
    elif "colorROJO" in clases:
        equipo = "rojo"
    else:
        equipo = "desconocido"
    
    # Extraer las coordenadas
    try:
        left = float(estilo.split("left:")[1].split("%")[0].strip())
        top = float(estilo.split("top:")[1].split("%")[0].strip())
    except:
        left, top = None, None
    
    data.append({
        "equipo": equipo,
        "resultado": resultado,
        "left_pct": left,
        "top_pct": top
    })

# Crear el DataFrame
df_tiros = pd.DataFrame(data)

df_tiros.head(20)


,equipo,resultado,left_pct,top_pct
0,azul,fallado,36.72,43.75
1,azul,fallado,36.72,36.68
2,azul,acertado,38.77,47.20
3,azul,fallado,26.07,36.35
4,rojo,acertado,85.55,49.01
5,azul,acertado,36.62,35.53
6,rojo,fallado,82.23,56.09
7,azul,fallado,35.35,70.89
8,azul,fallado,36.13,48.03
9,rojo,acertado,62.99,60.86
